In [1]:
import pandas as pd
import re
import json

In [4]:
df = pd.read_csv('python.csv')
df.head(2)

,property,jobId,jobType,jobName,jobTags,jobNumString,workAreaCode,jobAreaCode,jobAreaString,jobAreaLevelDetail,...,companyHref,allowChatOnline,ctmId,term,termStr,landmarkId,landmarkString,jobScheme,coId,hrActiveStatusGreen
0,"{""isInitiative"":""是"",""pageCode"":""sou|sou|soulb""...",148529708,0,Python全栈开发工程师,"['无需经验', '大专', 'python', '全栈', '机器视觉', '五险一金',...",NaN,20900,20000,上海·杨浦区,"{'provinceString': '上海', 'cityString': '上海', '...",...,https://jobs.51job.com/all/coVTFWNlExVmpTP1M3B...,False,1359547,0,全职,NaN,NaN,qiancheng://home/show_job_info?jobid=148529708...,2432975,NaN
1,"{""isInitiative"":""是"",""pageCode"":""sou|sou|soulb""...",148889136,0,python（爬虫）工程师,"['2年', '硕士', 'mysql', '数据爬取', 'python', 'cooki...",NaN,10800,10000,北京·海淀区,"{'provinceString': '北京', 'cityString': '北京', '...",...,https://jobs.51job.com/all/coBGVUOF47DjkOagZjB...,False,6889586,0,全职,3681.0,五道口,qiancheng://home/show_job_info?jobid=148889136...,7869166,NaN


In [47]:
'''
jobName: 岗位名称
provinceString: 省份
cityString: 城市名
max_salary: 最高工资
min_salary: 最低工资
workYearString: 工作经验
degreeString: 学历
fullCompanyName: 公司名
companyTypeString: 公司类型
companySizeString: 公司人数
termStr: 办公类型
jobTags: 标签
'''

'\njobName: 岗位名称\nprovinceString: 省份\ncityString: 城市名\nmax_salary: 最高工资\nmin_salary: 最低工资\nworkYearString: 工作经验\ndegreeString: 学历\nfullCompanyName: 公司名\ncompanyTypeString: 公司类型\ncompanySizeString: 公司人数\ntermStr: 办公类型\njobTags: 标签\n'

In [43]:
wan = 10000
qian = 1000
def clear_data(data):
    try: 
        if '万/年' in data:
            min_salary, max_salary = data.replace('万/年', '').split('-')
            data_list = [float(min_salary) * wan, float(max_salary) * wan] 
        elif ('千' in data) and ('万' in data) and ('薪' in data):
            xin_list = data.replace('万', '').replace('千', '').replace('薪', '').split('·')
            # 根据多少薪计算平均每月领到工资的比例
            number = float(xin_list[1]) / 12
            min_salary, max_salary = xin_list[0].split('-')
            data_list = [float(min_salary) * qian * number, float(max_salary) * wan * number] 
        elif ('千' in data) and ('薪' in data):
            xin_list = data.replace('千', '').replace('薪', '').split('·')
            # 根据多少薪计算平均每月领到工资的比例
            number = float(xin_list[1]) / 12
            min_salary, max_salary = xin_list[0].split('-')
            data_list = [float(min_salary) * qian * number, float(max_salary) * qian * number] 
        elif ('万' in data) and ('薪' in data):
            xin_list = data.replace('万', '').replace('薪', '').split('·')
            # 根据多少薪计算平均每月领到工资的比例
            number = float(xin_list[1]) / 12
            min_salary, max_salary = xin_list[0].split('-')
            data_list = [float(min_salary) * wan * number, float(max_salary) * wan * number] 
        elif ('千' in data) and ('万' in data):
            min_salary, max_salary = data.replace('万', '').replace('千', '').split('-')
            data_list = [float(min_salary) * qian, float(max_salary) * wan]
        elif '元/天' in data:
            salary = data.replace('元/天', '')
            data_list = [float(salary) * 30, float(salary) * 30]
        elif '千/天' in data:
            salary = data.replace('千/天', '')
            data_list = [float(salary) * 30 * qian, float(salary) * 30 * qian] 
        elif '万' in data:
            min_salary, max_salary = data.replace('万', '').split('-')
            data_list = [float(min_salary) * wan, float(max_salary) * wan] 
        elif '千' in data:
            min_salary, max_salary = data.replace('千', '').split('-')
            data_list = [float(min_salary) * qian, float(max_salary) * qian]
    except Exception as e:
        print(data)
        data_list = [3000, 4000]
    return data_list

provideSalary = df.provideSalaryString.apply(clear_data)
# 最低工资
df['min_salary'] = [int(i[0]) for i in provideSalary]
# 最高工资
df['max_salary'] = [int(i[1]) for i in provideSalary]
# 平均工资
df['mean_salary'] = [round((int(i[0])+int(i[1]))/2, 2) for i in provideSalary]

In [112]:
# 岗位平均薪资情况
# 按岗位名分类求平均
job_map_data = df.groupby('jobName')['min_salary', 'max_salary'].mean().round(2).reset_index().values
# 岗位名数组
job_salary_name = job_map_data[:, 0].tolist()
# 岗位最低工资数组
job_min_salary = job_map_data[:, 1].tolist()
# 岗位最高工资数组
job_max_salary = job_map_data[:, 2].tolist()
job_max_salary

C:\Users\wzj\AppData\Local\Temp\ipykernel_1676\3376837783.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  job_map_data = df.groupby('jobName')['min_salary', 'max_salary'].mean().round(2).reset_index().values


[150000.0,
 8000.0,
 35000.0,
 20000.0,
 21000.0,
 58333.0,
 20000.0,
 43333.0,
 26249.5,
 15000.0,
 28166.0,
 15166.0,
 30000.0,
 35000.0,
 15000.0,
 15000.0,
 25000.0,
 31250.0,
 15000.0,
 10833.0,
 43333.0,
 25000.0,
 21666.0,
 20000.0,
 13000.0,
 27083.0,
 18000.0,
 16249.0,
 12000.0,
 32499.0,
 30000.0,
 6500.0,
 19500.0,
 8000.0,
 18416.0,
 25000.0,
 18000.0,
 9000.0,
 21666.0,
 35000.0,
 17500.0,
 30000.0,
 18000.0,
 19500.0,
 35000.0,
 25000.0,
 20000.0,
 40000.0,
 17333.0,
 30000.0,
 13000.0,
 26000.0,
 23000.0,
 16811.57,
 9000.0,
 30000.0,
 37916.0,
 24000.0,
 28749.5,
 22000.0,
 20000.0,
 20000.0,
 20000.0,
 20000.0,
 6000.0,
 22000.0,
 15000.0,
 18000.0,
 20000.0,
 15000.0,
 15000.0,
 15000.0,
 15000.0,
 21000.0,
 19500.0,
 20000.0,
 40000.0,
 7500.0,
 15000.0,
 12000.0,
 16416.5,
 8000.0,
 16805.44,
 40000.0,
 20000.0,
 20020.5,
 26369.56,
 29166.0,
 7666.67,
 12000.0,
 25000.0,
 10000.0,
 16191.85,
 20000.0,
 10000.0,
 15000.0,
 26000.0,
 20000.0,
 27000.0,
 24916.0,
 17

In [83]:
# 提取城市名
def get_cityString(data):
    try:
        data = eval(data)
        return data['cityString']
    except Exception as e:
        print(data)
# 添加列
df['cityString'] = df.jobAreaLevelDetail.apply(get_cityString)

In [94]:
city_map_data = df.groupby('cityString')[
        'mean_salary'].mean().round(2).reset_index().values
city_name = city_map_data[:, 0].tolist()
city_values = city_map_data[:, 1].tolist()

In [113]:
# 热门岗位
job_value_counts = df.jobName.value_counts()
job_name = job_value_counts.index.to_list()[:10][::-1]
job_values =  job_value_counts.to_list()[:10][::-1]
len(job_value_counts)

475

In [102]:
# 学历占比
degree_dict = [{'value': v, 'name': n} for v, n in zip(df.degreeString.value_counts(), df.degreeString.value_counts().index)] 
# 工作经验占比
work_year_dict = [{'value': v, 'name': n} for v, n in zip(df.workYearString.value_counts(), df.workYearString.value_counts().index)] 
# 公司类型占比
company_type_dict = [{'value': v, 'name': n} for v, n in zip(df.companyTypeString.value_counts(), df.companyTypeString.value_counts().index)] 
degree_dict

[{'value': 638, 'name': '本科'},
 {'value': 294, 'name': '大专'},
 {'value': 52, 'name': '硕士'},
 {'value': 2, 'name': '中技/中专'},
 {'value': 1, 'name': '高中'}]

In [103]:
# 工作经验占比
work_year_dict = [{'value': v, 'name': n} for v, n in zip(df.workYearString.value_counts(), df.workYearString.value_counts().index)] 
work_year_dict

[{'value': 329, 'name': '3-4年'},
 {'value': 223, 'name': '2年'},
 {'value': 198, 'name': '1年'},
 {'value': 137, 'name': '无需经验'},
 {'value': 100, 'name': '5-7年'},
 {'value': 9, 'name': '8-9年'},
 {'value': 3, 'name': '10年以上'}]

In [104]:
# 公司类型占比
company_type_dict = [{'value': v, 'name': n} for v, n in zip(df.companyTypeString.value_counts(), df.companyTypeString.value_counts().index)] 
company_type_dict

[{'value': 673, 'name': '民营'},
 {'value': 87, 'name': '已上市'},
 {'value': 60, 'name': '外资（非欧美）'},
 {'value': 59, 'name': '国企'},
 {'value': 51, 'name': '合资'},
 {'value': 47, 'name': '外资（欧美）'},
 {'value': 8, 'name': '事业单位'},
 {'value': 7, 'name': '创业公司'}]

In [ ]:
df.jobName